In [1]:
!pip install -q transformers datasets accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
import torch
import os

print("🔧 Starting LoRA fine-tuning script...")

# --- Settings ---
model_name = "TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0"
dataset_path = "/kaggle/input/medidata"
dataset_file = "dataset.json"
output_dir = "/kaggle/working/tinyllama-lora-finetuned"

print(f"📁 Model: {model_name}")
print(f"📂 Dataset path: {os.path.join(dataset_path, dataset_file)}")
print(f"💾 Output will be saved to: {output_dir}")

# --- Load tokenizer and base model ---
print("📦 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

print("📦 Loading base model in 4-bit...")
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
)

# --- Prepare model for LoRA ---
print("🛠️ Preparing model for LoRA training...")
base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

print("🔌 Applying LoRA configuration...")
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# --- Load dataset (Alpaca format) ---
print("📄 Loading dataset...")
raw_dataset = load_dataset("json", data_files=os.path.join(dataset_path, dataset_file))["train"]
print(f"📊 Original dataset size: {len(raw_dataset)} samples")

# --- Reduce to 3,000 samples ---
raw_dataset = raw_dataset.shuffle(seed=42).select(range(63000))
print(f"📉 Reduced dataset size: {len(raw_dataset)} samples")

# --- Format dataset ---
print("✍️ Tokenizing dataset...")

def format_alpaca(example):
    instruction = example["instruction"]
    input_text = example["input"]
    output_text = example["output"]

    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"

    return {
        "input_ids": tokenizer(prompt + output_text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")["input_ids"][0],
        "labels": tokenizer(output_text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")["input_ids"][0]
    }

tokenized_dataset = raw_dataset.map(format_alpaca, num_proc=4)
print("✅ Dataset tokenization complete.")

# --- Training ---
print("🚀 Starting training...")

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    logging_dir=f"{output_dir}/logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="no",
    save_total_limit=1,
    learning_rate=2e-4,
    fp16=True,
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
print("✅ Training complete.")

# --- Save LoRA model ---
print("💾 Saving model and tokenizer...")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ LoRA fine-tuned TinyLLaMA model saved to: {output_dir}")


🔧 Starting LoRA fine-tuning script...
📁 Model: TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0
📂 Dataset path: /kaggle/input/medidata/dataset.json
💾 Output will be saved to: /kaggle/working/tinyllama-lora-finetuned
📦 Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

Map (num_proc=4):   0%|          | 0/63000 [00:00<?, ? examples/s]

✅ Dataset tokenization complete.
🚀 Starting training...


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-b15d2fbff83e>:100: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/pytho

Step,Training Loss
10,11.244700
20,10.460100
30,9.643800
40,9.322900
50,9.036500
60,9.164700
70,8.838400
80,9.138800
90,9.156100
100,8.851100


✅ Training complete.
💾 Saving model and tokenizer...
✅ LoRA fine-tuned TinyLLaMA model saved to: /kaggle/working/tinyllama-lora-finetuned
